---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

[Certificate](https://www.coursera.org/account/accomplishments/verify/EW5LACRXBUNY)

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

pd.options.display.max_columns = None

def blight_model():
    def preprocessCSV():
        df1 = pd.read_csv("train.csv", encoding="latin1")
        df2 = pd.read_csv("test.csv", encoding="latin1")
        
        df1 = df1[["agency_name","inspector_name","zip_code","ticket_issued_date","hearing_date",
                     "violation_code","fine_amount","admin_fee","state_fee","late_fee","discount_amount","clean_up_cost",
                     "judgment_amount","compliance"]]
        
        df2 = df2[["agency_name","inspector_name","zip_code","ticket_issued_date","hearing_date",
                     "violation_code","fine_amount","admin_fee","state_fee","late_fee","discount_amount","clean_up_cost",
                     "judgment_amount"]]
        df2["compliance"] = 1
        
        filter1 = df1["compliance"] >= 0
        df1 = df1[filter1]
        
        limiter = len(df1)
        
        df = pd.concat([df1,df2]).reset_index(drop=True)
            
        df["hearing_day"] = [-1 if type(x) == float else datetime(int(x[:4]),int(x[5:7]),
                                                                 int(x[8:10])).weekday() for x in df["hearing_date"]]
        df["hearing_month"] = [-1 if type(x) == float else int(x[5:7]) for x in df["hearing_date"]]
        df["hearing_hour"] = [-1 if type(x) == float else int(x[11:13]) for x in df["hearing_date"]]
        df["hearing_minute"] = [-1 if type(x) == float else int(x[14:16]) for x in df["hearing_date"]]

        df["ticket_issued_day"] = [-1 if type(x) == float else datetime(int(x[:4]),int(x[5:7]),
                                                                 int(x[8:10])).weekday() for x in df["ticket_issued_date"]]
        df["ticket_issued_month"] = [-1 if type(x) == float else int(x[5:7]) for x in df["ticket_issued_date"]]
        df["ticket_issued_hour"] = [-1 if type(x) == float else int(x[11:13]) for x in df["ticket_issued_date"]]
        df["ticket_issued_minute"] = [-1 if type(x) == float else int(x[14:16]) for x in df["ticket_issued_date"]]

        df = df.drop(labels=["ticket_issued_date","hearing_date"], axis=1)
        df["zip_code"] = [float(str(0)+"".join([i for i in x if i.isdigit()])) if type(x) is str else float(x) for x in df["zip_code"]]

        df = pd.get_dummies(df)
        df = df.fillna(-1)
        y = df["compliance"]
        X = df.drop(labels="compliance", axis=1)
        
        y=y[:limiter]
        X_train=X[:limiter]
        X_test=X[limiter:]
                
        return (X_train,X_test,y)
    
    X_train,X_test,y= preprocessCSV()
    scaler = MinMaxScaler()
    X_train_scalled = scaler.fit_transform(X_train)
    
    model = RandomForestClassifier(n_estimators = 150).fit(X_train_scalled,y)
    
    X_test_scalled = scaler.transform(X_test)
    
    probas = model.predict_proba(X_test_scalled)
    ticketIds = pd.read_csv("test.csv", encoding="latin1")
    ticketIds = list(ticketIds["ticket_id"])
    proba = list(probas[:,1])
    
    return pd.Series(data=proba, index=ticketIds)# Your answer here

In [19]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


284932    0.097181
285362    0.026863
285361    0.232139
285338    0.001131
285346    0.178789
285345    0.001131
285347    0.138000
285342    0.437956
285530    0.026667
284989    0.086667
285344    0.060503
285343    0.046754
285340    0.087105
285341    0.040503
285349    0.126667
285348    0.020000
284991    0.086667
285532    0.101846
285406    0.000000
285001    0.008602
285006    0.013541
285405    0.020000
285337    0.001250
285496    0.056293
285497    0.028572
285378    0.026667
285589    0.126667
285585    0.040120
285501    0.043367
285581    0.014358
            ...   
376367    0.070556
376366    0.053041
376362    0.053041
376363    0.067287
376365    0.070556
376364    0.053041
376228    0.046281
376265    0.030033
376286    0.362342
376320    0.117532
376314    0.042007
376327    0.313953
376385    0.257531
376435    0.300053
376370    0.316155
376434    0.097821
376459    0.175264
376478    0.083457
376473    0.033727
376484    0.015382
376482    0.054855
376480    0.

In [1]:

"""import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


pd.options.display.max_columns = None

def preprocessCSV(name):
    df = pd.read_csv(name, encoding="latin1")
    df = df[["agency_name","inspector_name","zip_code","ticket_issued_date","hearing_date",
             "violation_code","fine_amount","admin_fee","state_fee","late_fee","discount_amount","clean_up_cost",
             "judgment_amount","compliance"]]
    filter1 = df["compliance"] >= 0
    df = df[filter1]

    df["hearing_day"] = [-1 if type(x) == float else datetime(int(x[:4]),int(x[5:7]),
                                                             int(x[8:10])).weekday() for x in df["hearing_date"]]
    df["hearing_month"] = [-1 if type(x) == float else int(x[5:7]) for x in df["hearing_date"]]
    df["hearing_hour"] = [-1 if type(x) == float else int(x[11:13]) for x in df["hearing_date"]]
    df["hearing_minute"] = [-1 if type(x) == float else int(x[14:16]) for x in df["hearing_date"]]

    df["ticket_issued_day"] = [-1 if type(x) == float else datetime(int(x[:4]),int(x[5:7]),
                                                             int(x[8:10])).weekday() for x in df["ticket_issued_date"]]
    df["ticket_issued_month"] = [-1 if type(x) == float else int(x[5:7]) for x in df["ticket_issued_date"]]
    df["ticket_issued_hour"] = [-1 if type(x) == float else int(x[11:13]) for x in df["ticket_issued_date"]]
    df["ticket_issued_minute"] = [-1 if type(x) == float else int(x[14:16]) for x in df["ticket_issued_date"]]

    df = df.drop(labels=["ticket_issued_date","hearing_date"], axis=1)
    df["zip_code"] = [float(str(0)+"".join([i for i in x if i.isdigit()])) if type(x) is str else float(x) for x in df["zip_code"]]

    df = pd.get_dummies(df)
    df = df.fillna(-1)
    y = df["compliance"]
    X = df.drop(labels="compliance", axis=1)
    
    return (X,y)

X,y = preprocessCSV("readonly/train.csv")

batch_size = len(y)
train_size = batch_size - 1000

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=train_size)
scaler = MinMaxScaler()

X_train_scalled = scaler.fit_transform(X_train)
X_test_scalled = scaler.transform(X_test)"""


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
#print(X_train_scalled)
#print(X_test_scalled)
#print(y_train.iloc[int(batch_size/2)])

#X_train_scalled = X_train_scalled[:int(batch_size/10),:] 
#y_train = y_train.iloc[:int(batch_size/10)]

In [2]:
"""from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

print("fitting model...")
#model = KNeighborsClassifier(n_neighbors = 10).fit(X_train_scalled,y_train)
#model = LogisticRegression(penalty="l1", C=1000).fit(X_train_scalled,y_train)
#model = SVC(kernel = "linear",max_iter=100).fit(X_train_scalled,y_train)
#model = DecisionTreeClassifier(max_depth = 140).fit(X_train_scalled,y_train)
#model = RandomForestClassifier(n_estimators = 150).fit(X_train_scalled,y_train)
clasifier = RandomForestClassifier()
grid_values = {"n_estimators":[1,5,10,20,30,40,50,70,80,100,120,150]}
model = GridSearchCV(clasifier, param_grid = grid_values, scoring = 'roc_auc').fit(X_train_scalled,y_train)
print("Model fitted!")
print("Predicting...")
y_pred = model.predict(X_test_scalled)
print("Predicting...")
y_pred_train = model.predict(X_train_scalled)
print(roc_auc_score(y_test,y_pred))
"""

fitting model...
Model fitted!
Predicting...
Predicting...
0.615950817932


In [3]:
"""from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))
print(accuracy_score(y_train,y_pred_train))
#print(model.tree_.max_depth)
print(model.best_estimator_)
print(model.best_score_)
#print(y_pred[10])
#print(model.predict_proba(X_test_scalled)[:,1])"""

0.939
0.990854733132
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
0.788653052761


In [13]:
"""df1 = pd.DataFrame({"A": ["A0", "A1", "A2", "A3"],"B": ["B0", "B1", "B2", "B3"],"C": ["C0", "C1", "C2", "C3"],
                    "D": ["D0", "D1", "D2", "D3"]})
df2 = pd.DataFrame({"A": ["A0", "A1", "A2", "A3"],"B": ["B0", "B1", "B2", "B3"],"C": ["C0", "C1", "C2", "C3"],
                    "D": ["D0", "D1", "D2", "D3"]})
df3 = pd.concat([df1,df2]).reset_index(drop=True)

limit = len(df1)

df1_new = df3[:limit]

print(df1)
print(df2)
print(df3)

print(df1_new)


#print(df3[0])"""

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
4  A0  B0  C0  D0
5  A1  B1  C1  D1
6  A2  B2  C2  D2
7  A3  B3  C3  D3
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
